# Wordle Solver

In [17]:
# import functions
from wordle_helpers import *

In [18]:
# help(main)

In [19]:
# Main function that returns a list of possible words
green_letters = 'L'
green_positions = [3,]
yellow_letters = 'EO'
yellow_positions = [5,2,]
bad_letters = 'CRANDTS'

# Main function that returns a list of possible words
words = main(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters)
print(words)

['below', 'hello']


In [20]:
# Repetitive letters
letters_df = repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

Letters  Count
      L      3
      E      2
      O      2
      B      1
      W      1
      H      1


#### Good words:
SALET, CRANE, SHOUT, AUDIO, SLICE, SLICK, SPLIT, STUDY, INCUR, CHOIR, CROWN, CROWD, CRONE, POISE, OLIVE, DUNCE, BINGO, BISON, MIDST, SPOUT

In [21]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'LTSD'
yellow_positions = []
bad_letters = ''

# Main function that returns a list of possible words
words = main(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    bad_letters,
    wordle_list_path="wordle-possible-words",
)
print(words)

['dalts', 'delts', 'dolts']


In [22]:
# help(score)

In [23]:
df = load_data(file_name='wordle_ranking.csv')
df

,Names,Games_Won
0,<<<<<<< Updated upstream,NaN
1,Murilo,125.0
2,=======,NaN
3,Murilo,124.0
4,>>>>>>> Stashed changes,NaN
5,Barbara,53.0
6,Draw,118.0


In [24]:
# Update score
score_df = score()
score_df

,Names,Games_Won
1,Murilo,125.0
3,Murilo,124.0
5,Barbara,53.0
6,Draw,118.0


In [25]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo      125.0
 Murilo      124.0
Barbara       53.0
   Draw      118.0


In [26]:
# help(reset_score)

#### Set custom score

In [27]:
# help(set_score)

In [28]:
# # Set custom score
# df = set_score(m_score=107, b_score=41, draw_score=105)
# df

#### Reset score

In [29]:
## Reset score
# reset_score()